In [1]:
import pandas as pd
import numpy as np
import mailbox
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import OneClassSVM
from scipy.sparse import hstack
import scipy

In [2]:
trainfile = '/content/drive/MyDrive/groupib/train.mbox'
testfile = '/content/drive/MyDrive/groupib/test.mbox'

## Парсинг данных

In [3]:
def read_mbox(mb_file):
    mb = mailbox.mbox(mb_file)
    mbox_dict = {}

    for i, mail in enumerate(mb):
        mbox_dict[i] = {}
        for header in mail.keys():
            mbox_dict[i][header] = mail[header]
        mbox_dict[i]['Text'] = mail.get_payload()
    df = pd.DataFrame.from_dict(mbox_dict, orient='index')
    return df

In [4]:
def read_msg(mb_msg):
    if type(mb_msg) == list:
        mb = mailbox.mboxMessage(mb_msg[0])
        return mb.get_payload()
    else:
        return mb_msg

In [5]:
train = read_mbox(trainfile)
test = read_mbox(testfile)
test['Text'] = test['Text'].apply(read_msg)

## Препроцессинг

In [6]:
def subject_cleaner(text):
  text = text.str.strip()
  text = text.str.lower()
  return text

In [7]:
def body_cleaner(text):
    # text = text.str.replace(r'<[^>]+>', '')
    # text = text.str.replace(r'{[^}]+}', '')
    text = text.str.replace(r'#message', '')
    text = text.str.replace(r'\n{1,}', '')
    text = text.str.replace(r'={1,}', ' ')
    text = text.str.replace(r'-{2,}', ' ')
    text = text.str.replace(r'\*{1,}', ' ')
    text = text.str.replace(r'&nbsp{1,}', ' ')
    text = text.str.replace(r'\t', ' ')
    text = text.str.replace(r'\s{1,}', ' ')
    text = text.str.strip()
    text = text.str.lower()
    return text

In [8]:
train['Subject'] = subject_cleaner(train['Subject'])
test['Subject'] = subject_cleaner(test['Subject'])
train['Text'] = body_cleaner(train['Text'])
test['Text'] = body_cleaner(test['Text'])

In [9]:
train['Content-Type'] = train['Content-Type'].apply(lambda x: x.lower().split(';')[0])

In [10]:
test['Content-Type'] = test['Content-Type'].str.lower().str.strip()
test['Content-Type'] = test['Content-Type'].apply(lambda x: str(x).split(';')[0])
test['Content-Type'][test['Content-Type']=='nan'] = ''
test['Content-Type'][test['Content-Type']==''] = 'no_type'
test['Content-Type'][test['Content-Type'] == 'text/html content-transfer-encoding: 8bit\\r\\n'] = 'text/html'

In [11]:
#нераспарсеная почта
test['Text'] = test['Text'].fillna('email')

In [12]:
test['Subject'] = test['Subject'].fillna('')

## Токенизация

In [13]:
tfidf_word = TfidfVectorizer(
    #norm='l2',
    analyzer='word',
    ngram_range=(1, 3),
    )
tfidf_word.fit(train['Text'])
train_w = tfidf_word.transform(train['Text'])
test_w = tfidf_word.transform(test['Text'])

In [14]:
tfidf_char = TfidfVectorizer(
    analyzer='char_wb',
    ngram_range=(2, 7),
    )
tfidf_char.fit(train['Text'])
train_ch = tfidf_char.transform(train['Text'])
test_ch = tfidf_char.transform(test['Text'])

In [15]:
train_data = hstack([train_w,train_ch])
test_data = hstack([test_w, test_ch])

## Модель

In [16]:
%%time
clf = OneClassSVM().fit(train_data)

In [17]:
%%time
pred = clf.predict(test_data)

In [18]:
# я меняю здесь местами 0 и 1 (это сделано специально, чтобы сделать микс с баллами по чеку, см.ниже)
# если хотите посмотреть скор по тексту, удалите эту запись и добавьте pred[pred == -1] = 0
pred[pred == 1] = 0
pred[pred == -1] = 1

In [20]:
test['Predict_model'] = pred

## Дополнительный check для теста

### тест отправителя From
- enron - 0, 
- внешние - 1

In [21]:
test['check_email'] = test['From'].str.contains('~@enron') * 1

### тест на наличие warning words в теме письма
- 1 слово - 0.5 баллов, 
- 2 слова и более - 1 бал, 
- 0 баллов нет

In [22]:
wwords = ['payment', 'urgent', 'bank', 'account', 'access', 'block', 'limit', 'confirm', 'important',
'password', 'require', 'file', 'download', 'request', 'security', 'validat', 'suspend', 'verificat',
'update', 'cash', 'fraud', 'error', 'alert', 'lock', 'card', 'bill', 'official', 'online', 'secure',
'profile', 'modif', 'deposit', 'offer', 'verif', 'inquiry', 'free', 'unusual', 'identif', 
]

In [23]:
def check_sub(x):
  cnt = 0
  x = re.sub(r'[^A-Za-z]', ' ', x)
  for i in x.split():
    if i in wwords:
     cnt += 1
  if cnt == 0:
    return 0
  if cnt == 1:
    return 0.5
  if cnt > 1:
    return 1

In [24]:
test['check_subject'] = test['Subject'].apply(check_sub)

### тест на наличие warning words в теле письма
- 1 слово - 0.5 баллов, 
- 2 слова и более - 1 бал, 
- 0 баллов нет

In [25]:
test['check_body'] = test['Text'].apply(check_sub)

### тест на наличие http ссылок, за исключением номинального имени сайта, в теле письма
- есть - 1 бал,
- нет - 0 баллов

In [26]:
test['check_text_http'] = test['Text'].str.contains(r'(http|https):\/\/.+?(?=\/)\/\w') * 1

/usr/local/lib/python3.7/dist-packages/pandas/core/strings.py:2001: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


### тест столбца Content-Type
- чистый текст - 0 балов,
- ссылки, вставки и тд - 3 балла

In [27]:
test['check_content_type']=test['Content-Type'].apply(lambda x: 0 if x == 'text/plain'else 3)

### сумируем check

In [28]:
test['sum_check'] = test['check_email'] + test['check_subject'] + test['check_body'] + test['check_text_http']+test['check_content_type'] 

### Миксуем

In [40]:
test['sum_score'] = test['Predict_model'] + test['sum_check']

In [41]:
test['Label']= test['sum_score'].apply(lambda x: 0 if x >= 4 else 1)

## Выгрузка

In [33]:
test[['X-UID', 'Label']].to_csv('Result.csv', index=False)